# Quality check Assignment(1) 

## Basics

Import packages first

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Figuring out the limits 

For filtering the colors in processed and raw food, i used the following function. Running the cell is optional.

In [2]:
def nothing(x):
    pass
def filter():
    #cap = cv2.VideoCapture(0)
    cv2.namedWindow('HSV')
    cv2.createTrackbar('low_H', 'HSV', 0, 180, nothing)
    cv2.createTrackbar('low_S', 'HSV', 0, 255, nothing)
    cv2.createTrackbar('low_V', 'HSV', 0, 255, nothing)
    cv2.createTrackbar('high_H', 'HSV', 180, 180, nothing)
    cv2.createTrackbar('high_S', 'HSV', 255, 255, nothing)
    cv2.createTrackbar('high_V', 'HSV', 255, 255, nothing)
    frame = cv2.imread('rawproduct.jpg', -1)
    while(True):
        #_, frame = cap.read()
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        low_H = cv2.getTrackbarPos('low_H', 'HSV')
        low_S = cv2.getTrackbarPos('low_S', 'HSV')
        low_V = cv2.getTrackbarPos('low_V', 'HSV')
        high_H = cv2.getTrackbarPos('high_H', 'HSV')
        high_S = cv2.getTrackbarPos('high_S', 'HSV')
        high_V = cv2.getTrackbarPos('high_V', 'HSV')
        low_limit = np.array([low_H,low_S,low_V])
        high_limit = np.array([high_H,high_S,high_V])
        mask = cv2.inRange(hsv, low_limit, high_limit)
        #print(low_limit.dtype)
        #print(low_limit)
        #print(high_limit)
        cv2.imshow('HSV',mask)
        cv2.imshow('frame', frame)
        cv2.imshow('mask', mask)
        k=cv2.waitKey(1)
        if (k==27):
            break
    cv2.destroyAllWindows()

## Limits checking

Checking the limits of the function has been done by this function, again running the cell is optional.

In [3]:
def checklimits(image):
    imglist = image.split()
    for img in imglist:
        
        frame = cv2.imread(img,cv2.IMREAD_COLOR)
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

#lower_raw = np.array([16,43,202])
#higher_raw = np.array([30,255,255])

        lower_raw = np.array([5,110,50])
        higher_raw = np.array([180,255,255])

        mask = cv2.inRange(frame, lower_raw, higher_raw)
        maskinv = cv2.bitwise_not(mask)
        res = cv2.bitwise_and(frame, frame, mask = mask)
        resinv = cv2.bitwise_and(frame, frame, mask = maskinv)
        print(lower_raw.dtype)
        cv2.imshow(img+' frame',frame)
        cv2.imshow(img+' mask', mask)
        cv2.imshow(img+' res', res)
        cv2.imshow(img+' resinv', resinv)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Image segmentation - The actual stuff begins here

Importing the product Image, run the cell before proceeding.

In [4]:
image = "processedproduct.png"
frame = cv2.imread(image, cv2.IMREAD_COLOR)

Range of the colors to create a mask. Run and Proceed

In [5]:
lower_prod=np.array([5,110,50])
higher_prod = np.array([180,255,255])

Converting to HSV and applying the mask.

In [6]:
#contour detection


#blurred = cv2.GaussianBlur(frame,(41,41),cv2.BORDER_DEFAULT)
#gray = cv2.cvtColor(blurred,cv2.COLOR_BGR2GRAY)

#_,threshold = cv2.threshold(gray,0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#cannyd = cv2.Canny(frame, 50,100)

hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv,lower_prod, higher_prod)
maskinv =  cv2.bitwise_not(mask)

Performing dilation followed by erosion (also known as opening) to separate the Chips.

In [7]:
#blurred = cv2.medianBlur(maskinv, 5)
kernel = np.ones((15,15), np.uint8)
#opening = cv2.morphologyEx(blurred,cv2.MORPH_CLOSE, kernel)
dilating =cv2.dilate(maskinv, kernel, iterations = 1)
opening = cv2.erode(dilating, kernel, iterations =1)


Finding all the contours on the mask obtained.

In [8]:
#contours, _ =cv2.findContours(blurred, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
contours, _ =cv2.findContours(opening, cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
#contours, _ = cv2.findContours(threshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
#contourscany,_ = cv2.findContours(cannyd, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
#print("Number of contours detected", len(contours))
#print("Number of canny contours",len(contourscany))

#cv2.drawContours(frame, contours,-1,(0,0,255),2)
#cv2.drawContours(frame, contourscany, -1,(0,0,255),2)
#cv2.imshow('threshold',threshold)

Drawing the useful contours, and storing their values such as bounding rectangles and area of contours

In [9]:
totcontour =0
rectlist=[]
rectpoints=(0,0,0,0)
font =cv2.FONT_HERSHEY_SIMPLEX
arealist=[]
#hull_list=[]
for contour in contours:
    area=cv2.contourArea(contour)
    arealist.append(area)
    if(area>50 and area<10000):
        cv2.drawContours(frame,contour,-1, (0,0,255),1)
        totcontour +=1
        cv2.putText(frame, str(totcontour),(rectpoints[0],rectpoints[1]),font,1,(0,255,255),1)
        rectpoints=cv2.boundingRect(contour)
        rectlist.append(rectpoints)
        #print(rectpoints)
        """hull = cv2.convexHull(contour,returnPoints = False)
        hull_list.append(hull)"""
        #print("new :",hull)
        #defects = cv2.convexityDefects(contour,hull)
        #print(defects)
        """major_defect=[]
        for defect in defects:
            if(defect[3]>20):
                major_defect.append(defect[2])
        major_defect=np.array(major_defect)
        if(len(major_defect)==2):
            diff=[ np.absolute( major_defect[0][i] - major_def[1][i] ) for i in range(2)]
            if(diff[0]>diff[1]):
                major_defect[1][1]=major_defect[0][1]                            
            else:
                major_defect[1][0]=major_defect[0][0] """
                
        #cv2.polylines(frame, hull, True,(255,0,0),5)
        
        #cv2.drawContours(frame,hull,-1,(255,0,0),6)        
        cv2.rectangle(frame,rectpoints,(255,0,0),1)

Displaying and storing the contours and the processed images

In [10]:
#print(len(hull_list))
#print("NUmber of cann contours =", totcontour)
cv2.imshow('frame',frame)
cv2.imwrite('frame.jpg',frame)
#cv2.imshow('blurred', blurred)
#cv2.imshow('gray', gray)
cv2.imshow('mask', mask)
cv2.imwrite('mask.jpg',mask)
cv2.imshow('maskinv',maskinv)
cv2.imwrite('maskinv.jpg',maskinv)
cv2.imshow('dilated',dilating)
cv2.imwrite('dilated.jpg',dilating)
cv2.imshow('opened', opening)
cv2.imwrite('opened.jpg',opening)
cv2.waitKey(0)
cv2.destroyAllWindows()

Running the above cell you'll see one of these pop up. 

![title](frame.jpg)


Hence the images have been segmented.(24/25 chips were detected)

## Extracting individual chips

In [11]:
newframe = cv2.imread(image, -1)
newres = cv2.bitwise_and(newframe, newframe, mask = mask)
cv2.imshow('res',newres)
cv2.waitKey(0)
cv2.destroyAllWindows()
roilist=[]
roinum=0
for rect in rectlist:
    roi=newres[rect[1]:rect[1]+rect[3],rect[0]:rect[0]+rect[2]]
    roinum+=1
    roilist.append(roi)
    #totroi.concatenate
    cv2.imshow("roi",roi)
    cv2.imwrite("roi"+str(roinum)+".jpg",roi)
    cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("new frame",newframe)
cv2.waitKey(0)
cv2.destroyAllWindows()

The first three detected chips are:
<img src="roi1.jpg" alt="region of interest of 1st chip" /><img src="roi2.jpg" alt="region of interest of 2nd chip" /><img src="roi3.jpg" alt="region of interest of 3rd chip" />

## Classifying the chips - Rejected/Accepted

This will be done on the following 2 categories:
<ul>
    <li>If the ratio of area of red color on the chip to the total area of red and white color is greater than 0.68 then it accepted otherwise it is rejected.</li>
    <li>If there is any other color on the chip then it is rejected.(the only color considered here is green because it is next most likely to be present) </li>
</ul>
<hr>

Initially every chip is considered to be accepted.

In [12]:
acceplist = np.ones((24,1),dtype=np.uint8)

## Area Calculation

The color limits used for area calculation

In [13]:
lower_red=np.array([0,175,105])
upper_red=np.array([40,255,255])

Calculating the area of white and red separately using limits found

In [14]:
newhsv = cv2.cvtColor(newframe, cv2.COLOR_BGR2HSV)
"""redmask = cv2.inRange(newhsv, lower_red, upper_red)
redmaskinv=cv2.bitwise_not(redmask)
for rect in rectlist:
    cv2.rectangle(redmask,rect,(255,255,255),2)
redmaskinv=cv2.bitwise_not(redmask)

cv2.imshow("red mask",redmask)
cv2.imshow("red mask inverse",redmaskinv)"""

arearedlist=[]
ratiolist=[]
for i in range(len(roilist)):
    roi = roilist[i]
    totA = arealist[i]
    rect = rectlist[i]
    roihsv=cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
    roimask = cv2.inRange(roihsv,lower_red,upper_red)
    roires = cv2.bitwise_and(roi,roi,mask=roimask)    
    controi, _ = cv2.findContours(roimask,cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(roires,controi, -1,(255,0,0),1)
    summ = 0
    for cont in controi:
        arroi = cv2.contourArea(cont)
        summ=summ+arroi
    arearedlist.append(summ)
    ratiolist.append(summ/totA)
    cv2.imshow("res roi",roires)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    #print("red area",summ)
    print("ratio",ratiolist[i])

"""cv2.imshow("frame",frame)
cv2.waitKey(0)
cv2.destroyAllWindows()"""

ratio 0.007254592692692274
ratio 0.5788408463661453
ratio 0.34898368164901233
ratio 1.0958049886621315
ratio 0.5914558914250463
ratio 1.0947965941343425
ratio 0.11264964661762585
ratio 0.526514704145506
ratio 1.4861451460885957
ratio 0.45107705497803025
ratio 0.5821450712957222
ratio 0.43964702174598175
ratio 0.01846772718961419
ratio 0.7452675510508272
ratio 0.33156460633873513
ratio 0.35251482538985285
ratio 0.001694915254237288
ratio 0.7250569476082005
ratio 0.5117222723174031
ratio 2.626797249426964
ratio 0.04691664084288813
ratio 0.008451957295373666
ratio 0.20975710168793743
ratio 0.006673436819962281


'cv2.imshow("frame",frame)\ncv2.waitKey(0)\ncv2.destroyAllWindows()'

# Detecting mossy/unhealthy chips

Green color is an indication of moss and is generally thought to be unhygenic<hr>Range for green color

In [15]:
sensitivity = 1
lower_green = np.array([60 - sensitivity, 100, 50])
upper_green = np.array([60 + sensitivity, 255, 255])

If green is detected, we'll be rejecting it with utmost priority.

In [16]:
isgreenlist=[]
for roi in roilist:
    roigreenmask = cv2.inRange(roi,lower_green, upper_green)
    #cv2.imshow("green roi mask", roigreenmask)
    if(cv2.countNonZero(roigreenmask)>10):
        isgreenlist.append(1)
    else:
        isgreenlist.append(0)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    

Rejecting the chips that don't fit the criteria.

In [17]:
for i in range(len(roilist)):
    if(isgreenlist[i]==0 or ratiolist[i]<0.68):
        acceplist[i]=0
#print(acceplist)

Printing this info on the frame.

In [18]:
for i in range(len(acceplist)):
    if(acceplist[i]==1):
        cv2.putText(frame,"Accepted",(rectlist[i][0]+rectlist[i][2]//2,rectlist[i][1]+rectlist[i][3]//2),font,0.75,(0,0,255),1)
    else:
        cv2.putText(frame,"Rejected",(rectlist[i][0]+rectlist[i][2]//2,rectlist[i][1]+rectlist[i][3]//2),font,0.75,(0,0,255),1)


Displaying the final Result(and storing).

In [19]:
cv2.imshow("frame", frame)
cv2.imwrite("result.jpg",frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

![title](result.jpg)

## Without the green color

(Temporary)The ranges found for the green color at present is buggy and requires further efforts. Hence the final result has been evaluated in a manner such that the test for green color has been neglected. The following cells will produce the actual result. (Printed in green) 

In [20]:
for i in range(len(roilist)):
    if(isgreenlist[i]==1 or ratiolist[i]<0.68):
        acceplist[i]=0
#print(acceplist)

In [21]:
for i in range(len(acceplist)):
    if(acceplist[i]==1):
        cv2.putText(frame,"Accepted",(rectlist[i][0]+rectlist[i][2]//2,rectlist[i][1]+rectlist[i][3]//2),font,0.5,(0,255,0),1)
    else:
        cv2.putText(frame,"Rejected",(rectlist[i][0]+rectlist[i][2]//2,rectlist[i][1]+rectlist[i][3]//2),font,0.5,(0,255,0),1)


In [22]:
cv2.imshow("frame", frame)
cv2.imwrite("resultorg.jpg",frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

![title](resultorg.jpg)